# Transfer Learning para o dataset VIPeR

Modelo treinado com os datasets Market-1501 e CUHK03

In [1]:
%load_ext autoreload
%autoreload 2

In [73]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch
import torch as tc
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision import datasets as tv_datasets
from torch.autograd import Variable
from  torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.pyplot as plt
%matplotlib inline
import torchvision.datasets as dset
import numpy as np
import cv2

from center_loss import CenterLoss
from frw import L2_penalty_with_constant
from reid_model import ReID_Net
from datasets import VIPeR
from custom_transforms import RandomTranslation
from evaluation import pairwise_squared_euclidian_distance, generate_query, \
                        get_topk_results, mapk, calculate_mAP
import time
import shutil

## Dataset, transformações de data augmentation e DataLoaders

In [5]:
data_transforms = {
    'train': transforms.Compose([
        RandomTranslation(0.1),   # translação em numpy array
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),    # range [0.0, 1.0]
        transforms.Lambda(lambda t: t/0.5 - 1)    # range [-1.0, 1.0]
    ]),
    'valid': transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda t: t/0.5 - 1)
    ])
}

In [74]:
datasets = {
    'train': VIPeR('../reid/datasets/viper_train_correct.npz', 
                            transform=data_transforms['train']),
    'valid': VIPeR('../reid/datasets/viper_train_correct.npz', train=False, transform=data_transforms['valid'])
}

batch_size = 128
dset_loaders = {
    'train': DataLoader(datasets['train'], batch_size=batch_size, shuffle=True, num_workers=4),
    'valid': DataLoader(datasets['valid'], batch_size=batch_size, shuffle=False, num_workers=4)
}

In [75]:
print('VIPeR:')
print('Treinamento:')
print('X:', datasets['train'].X.shape, datasets['train'].X.dtype, datasets['train'].X.min(), datasets['train'].X.max())
print('y:', datasets['train'].y.shape, datasets['train'].y.dtype, datasets['train'].y.min(), datasets['train'].y.max())
print('Validação:')
print('X:', datasets['valid'].X.shape, datasets['valid'].X.dtype, datasets['valid'].X.min(), datasets['valid'].X.max())
print('y:', datasets['valid'].y.shape, datasets['valid'].y.dtype, datasets['valid'].y.min(), datasets['valid'].y.max())

VIPeR:
Treinamento:
X: (432, 128, 48, 3) uint8 0 255
y: (432,) int64 0 215
Validação:
X: (200, 128, 48, 3) uint8 0 255
y: (200,) int64 0 99


In [77]:
print(len(dset_loaders['train']), len(dset_loaders['valid']))

4 2


# Carregando o modelo para Transfer Learning

O modelo carregado tem a última camada `classifier` e o buffer `centers` com dimensões incorretas para o uso no treinamento do VIPeR.

In [43]:
model_to_transfer = 'market1501_and_cuhk03_map_epoch_136.pth'
ckpt = torch.load(model_to_transfer)

In [25]:
num_centers = ckpt['state_dict']['centers'].size(0)
print(num_centers)

1918


In [33]:
num_classes = len(np.unique(datasets['train'].y))
print('Novo número de classes: ', num_classes)

Novo número de classes:  216


In [92]:
model = ReID_Net(num_centers)
model.load_state_dict(ckpt['state_dict'])
model.prepare_model_to_transfer(num_classes)

In [45]:
print('Novas dimensões dos centros: {}'.format(model.centers.size()))
print('Nova camada final: ', model.classifier)

## Dataset, transformações de data augmentation e DataLoaders

Novas dimensões dos centros: torch.Size([216, 512])
Nova camada final:  Linear (512 -> 216)


## Funções auxiliares

In [10]:
class MetricAverager(object):
    
    def __init__(self):
        self._n = 0
        self._mean = 0
        self._min = np.Inf
        
    def update(self, val):
        self._n += 1
        self._mean = (self._mean*(self._n - 1) + val) / self._n
        if val < self._min:
            self._min = val

    def clear(self):
        self._n = 0
        self._mean = 0
        self._min = np.Inf

    @property
    def mean(self):
        return self._mean

    @property
    def min(self):
        return self._min

    def __str__ (self):
        return 'MetricAverager(Mean: {:.5f}, n: {})'.format(self._mean, self._n)
        
    def __format__(self, format_spec):
        return format(self._mean, format_spec)

In [11]:
def save_checkpoint(state, is_best, filename='checkpoint.pth'):
    torch.save(state, filename)
    if is_best:
        import os.path as osp
        fname, ext = osp.splitext(filename)
        best_filename = '{}_best{}'.format(*osp.splitext(filename))
        shutil.copyfile(filename, best_filename)

In [12]:
def load_checkpoint(filename, model, optimizer):
    print('Carregando modelo {}'.format(filename))
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['state_dict'])
    scheduler = checkpoint.get('scheduler', None)
    epoch = checkpoint['epoch']
    best_val_loss = checkpoint['best_val_loss']
    train_history = checkpoint.get('history', [])
    
    print('Carregado.')
    return scheduler, epoch, best_val_loss, train_history

In [13]:
def train_epoch(model, optimizer, epoch, criterions, train_loader, val_loader, use_cuda,
                lambda_cl, beta_frw, query_splits, cuda_device=None):
    print("Training... Epoch = %d" % epoch)

    train_accum = 0
    n_train_samples = 0
    val_accum = 0
    n_val_samples = 0
    
    tlosses = {
        'center_loss': MetricAverager(),
        'softmax_loss': MetricAverager(),
        'total': MetricAverager(),
    }
    
    #train
    model.train(True)
    t0 = time.time()
    for i,(data, target) in enumerate(train_loader):
        
        if use_cuda:
            data = data.cuda()
            target = target.cuda()
        data, target = Variable(data), Variable(target)
        
        features, pred = model(data)
        
        # Softmax loss + center loss
        softmax_loss = criterions[0](pred, target)
        center_loss = lambda_cl * criterions[1](features, target)
        loss = softmax_loss + center_loss
        
        # Acrescenta a penalização L2 na camada FRW
        for (name, p) in model.named_parameters():
            if name == 'frw.weight' or name == 'fc_embeddings.3.weight':
                loss += beta_frw * criterions[2](p)
        
        _, preds = torch.max(pred.data, 1)
        correct = (preds == target.data).sum()
        train_accum += correct
        n_train_samples += target.size(0)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        tlosses['center_loss'].update(center_loss.data.cpu().numpy()[0])
        tlosses['softmax_loss'].update(softmax_loss.data.cpu().numpy()[0])
        tlosses['total'].update(loss.data.cpu().numpy()[0])
        
        t1 = time.time() - t0
        train_string = 'Epoch {}, Batch {}/{} ({:.1f}s): Center loss {:.4f}, Softmax loss {:.4f}, ' \
            'total {:.4f}'.format(epoch, i+1, len(train_loader), t1, tlosses['center_loss'], 
                                  tlosses['softmax_loss'], tlosses['total'])
        print(train_string, end='\r' if i < len(train_loader)-1 else '\n')
    
    train_acc = train_accum / n_train_samples
    print('Train accuracy: {:.2%}'.format(train_acc))
        
    # Validation: in the validation phase, we extract the features of all the validation images.
    # Then, we calculate the pairwise euclidian distance for all samples and calculate
    # the Mean Average Precision for the rankings in a Cross-camera search configuration: the query and
    # gallery sets have images from different camera views and, for each person identity, one image 
    # is randomly selected for each set. This is done for N query/gallery split configurations.
    
    t0 = time.time()
    model.train(False)
    
    val_feats = []
    val_pids = []
    
    valid_metrics = {
        'map': MetricAverager(),
    }
    
    for i, (data, target) in enumerate(val_loader):
        
        if use_cuda:
            data = data.cuda()
        data = Variable(data, volatile=True)

        features, pred = model(data)
        val_feats.append(features.data.cpu())
        
    val_feats = torch.cat(val_feats)
    
    # Calculates mAP for the validation set, as if it was a Query-Gallery set
    val_dataset = val_loader.dataset
    val_cam_ids = val_dataset.cam_ids
    y_val = val_dataset.y
    
    # Gallery: may or may not include distractors
    y_gallery = y_val
    gal_cam_ids = val_cam_ids
    
    for i, query_indices_by_cam in enumerate(query_splits):
        #query_indices_by_cam = generate_query(y_val, val_cam_ids)
        val_map = calculate_mAP(query_indices_by_cam, val_feats, y_gallery, gal_cam_ids)
        valid_metrics['map'].update(val_map)

        t1 = time.time() - t0
        print('Validation mAP (split {:>2}/{}) ({:.1f}s): {:.2%}'.format(i+1, len(query_splits),
                t1, valid_metrics['map'].mean), end='\r' if i+1 < len(query_splits) else '\n')
        
    print('-'*80)
        
    
    metrics = {
        'train': {
            'losses': {
                'center_loss': tlosses['center_loss'].mean,
                'softmax_loss': tlosses['softmax_loss'].mean,
                'total': tlosses['total'].mean,
            },
            'accuracy': train_acc,
        },
        'valid': {
            'map': valid_metrics['map'].mean
        }
    }
    
    return metrics

# Loop de treinamento

In [85]:
def train(epochs, model=None, resume=None, checkpoint_fn=None, **train_args):
    best_val_loss = None
    last_epoch = 0
    
    train_history = []
    
    use_cuda = train_args.get('use_cuda')
    num_classes = train_args.pop('num_classes', None)
    
    if model is None:
        if num_classes is None:
            raise ValueError('Deve-se informar o número de classes {`num_classes`}')
        model = ReID_Net(num_classes)
    
    if resume is not None:
        scheduler, last_epoch, best_val_loss, train_history = load_checkpoint(resume, model, optimizer4nn)
        num_classes = model.centers.size(0)
    
    # NLLLoss
    nllloss = nn.NLLLoss() #CrossEntropyLoss = log_softmax + NLLLoss
    # CenterLoss
    loss_weight = 1.0

    # FRW L2 penalty
    frw_l2_penalty = L2_penalty_with_constant(200)
    
    if use_cuda:
        nllloss = nllloss.cuda()
        model = model.cuda()
        frw_l2_penalty = frw_l2_penalty.cuda()
        
    
    alpha = train_args.pop('alpha')

    centerloss = CenterLoss(model, model.centers, num_classes, alpha)
    criterions = [nllloss, centerloss, frw_l2_penalty]

    # optimzer4nn
    lr = train_args.pop('lr')
    weight_decay = train_args.pop('weight_decay')
    optimizer4nn = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                                     lr=lr, weight_decay=weight_decay)
    
    # LR Scheduler
    scheduler_patience = train_args.pop('scheduler_patience', 35)
    scheduler_factor = train_args.pop('scheduler_factor', 0.1)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer4nn, mode='max',
                                               patience=scheduler_patience, 
                                               factor=scheduler_factor, verbose=True)
        
    max_val_map = best_val_loss or -np.Inf
    
    patience = train_args.pop('patience', 50)
    no_improve = 0
    
    print('train_args:', train_args, '\n')
    
    # Gera os splits de query e gallery que serão usados nas métricas de validação
    val_dataset = train_args['val_loader'].dataset
    query_splits = []
    for i in range(5):
        query_splits.append(generate_query(val_dataset.y, val_dataset.cam_ids))
    
    for epoch in range(last_epoch+1, epochs):

        metrics = train_epoch(model, optimizer4nn, epoch, criterions,
                              query_splits=query_splits, **train_args)
        
        scheduler.step(metrics['valid']['map'])
        train_history.append(metrics)
        
        # Save checkpoint and EarlyStop
        if metrics['valid']['map'] > max_val_map:
            print("Max. Valid. mAP / Epoch's Valid. mAP: {:.2%} | {:.2%}".format(
                max_val_map, metrics['valid']['map']))
            print('Salvando melhor modelo...')
            max_val_map = metrics['valid']['map']
            state = {
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'best_val_loss': max_val_map,
                'train_softmax_loss': metrics['train']['losses']['softmax_loss'],
                'history': train_history,
            }
            opts = {'is_best': False}
            if checkpoint_fn is not None:
                opts.update({'filename': checkpoint_fn.format(epoch+1)})
            save_checkpoint(state, **opts)
            no_improve = 0
            print('-'*80)
        else:
            no_improve += 1
            if no_improve > patience:
                print('Early stopping at epoch {}.'.format(epoch))
                break
                
    return model, train_history

# Primeira etapa: treinamento somente da última camada

Congela todos os parâmetros, exceto os da última camada

In [93]:
# len(model.parameters) == 53
for i, p in enumerate(model.parameters()):
    if i not in (51, 52):
        p.requires_grad = False

# Verifica que há apenas 2 parâmetros com requires_grad == True
for p in filter(lambda x: x.requires_grad, model.parameters()):
    print(p.size())

torch.Size([216, 512])
torch.Size([216])


### Parâmetros de treinamento

In [91]:
epochs = 500

train_params = {
    'lr': 0.0001,
    'weight_decay': 0.001,
    'lambda_cl': 0.01,
    'alpha': 0.5,
    'num_classes': len(np.unique(datasets['train'].y)),
    'train_loader': dset_loaders['train'],
    'val_loader': dset_loaders['valid'],
    'use_cuda': True,
    'beta_frw': 0.0001,
    'cuda_device': None,
    'patience': 50,
    'scheduler_patience': 20,
    'scheduler_factor': 0.1
}

In [94]:
try:
    model, train_history = train(epochs, model=model, 
                                 checkpoint_fn='transfer_viper_classifier_{}.pth', 
                                 **train_params)
    
except KeyboardInterrupt:
    print('Interrompendo...')

train_args: {'use_cuda': True, 'train_loader': <torch.utils.data.dataloader.DataLoader object at 0x7fd826cd2d68>, 'val_loader': <torch.utils.data.dataloader.DataLoader object at 0x7fd826cd24a8>, 'lambda_cl': 0.01, 'cuda_device': None, 'beta_frw': 0.0001} 

Training... Epoch = 1
Epoch 1, Batch 4/4 (0.3s): Center loss 4.2064, Softmax loss 5.5125, total 9.72135
Train accuracy: 0.69%
Validation mAP (split  5/5) (0.3s): 40.72%
--------------------------------------------------------------------------------
Max. Valid. mAP / Epoch's Valid. mAP: -inf% | 40.72%
Salvando melhor modelo...
--------------------------------------------------------------------------------
Training... Epoch = 2
Epoch 2, Batch 4/4 (0.4s): Center loss 2.0806, Softmax loss 5.4690, total 7.5520
Train accuracy: 1.16%
Validation mAP (split  5/5) (0.2s): 40.52%
--------------------------------------------------------------------------------
Training... Epoch = 3
Epoch 3, Batch 4/4 (0.4s): Center loss 1.3672, Softmax loss 5.

Validation mAP (split  5/5) (0.3s): 41.36%
--------------------------------------------------------------------------------
Training... Epoch = 30
Epoch 30, Batch 4/4 (0.4s): Center loss 0.9475, Softmax loss 4.4493, total 5.3993
Train accuracy: 18.75%
Validation mAP (split  5/5) (0.4s): 41.96%
--------------------------------------------------------------------------------
Epoch    29: reducing learning rate of group 0 to 1.0000e-05.
Training... Epoch = 31
Epoch 31, Batch 4/4 (0.5s): Center loss 0.9420, Softmax loss 4.4278, total 5.3722
Train accuracy: 19.68%
Validation mAP (split  5/5) (0.4s): 41.71%
--------------------------------------------------------------------------------
Training... Epoch = 32
Epoch 32, Batch 4/4 (0.4s): Center loss 0.9626, Softmax loss 4.4327, total 5.3978
Train accuracy: 18.29%
Validation mAP (split  5/5) (0.2s): 41.52%
--------------------------------------------------------------------------------
Training... Epoch = 33
Epoch 33, Batch 4/4 (0.3s): Center 

# Segunda etapa: fine-tuning da rede completa


In [99]:
# len(model.parameters) == 53
for i, p in enumerate(model.parameters()):
    p.requires_grad = True

# Verifica que há apenas 2 parâmetros com requires_grad == True
print('Número de camadas que serão otimizadas:', len(list(filter(lambda x: x.requires_grad, model.parameters()))))

Número de camadas que serão otimizadas: 53


### Parâmetros de treinamento

In [91]:
epochs = 500

train_params = {
    'lr': 0.0001,
    'weight_decay': 0.001,
    'lambda_cl': 0.01,
    'alpha': 0.5,
    'num_classes': len(np.unique(datasets['train'].y)),
    'train_loader': dset_loaders['train'],
    'val_loader': dset_loaders['valid'],
    'use_cuda': True,
    'beta_frw': 0.0001,
    'cuda_device': None,
    'patience': 50,
    'scheduler_patience': 20,
    'scheduler_factor': 0.1
}

In [100]:
try:
    model, train_history = train(epochs, model=model, 
                                 checkpoint_fn='transfer_viper_finetuning_{}.pth', 
                                 **train_params)

except KeyboardInterrupt:
    print('Interrompendo...')

train_args: {'use_cuda': True, 'train_loader': <torch.utils.data.dataloader.DataLoader object at 0x7fd826cd2d68>, 'val_loader': <torch.utils.data.dataloader.DataLoader object at 0x7fd826cd24a8>, 'lambda_cl': 0.01, 'cuda_device': None, 'beta_frw': 0.0001} 

Training... Epoch = 1
Epoch 1, Batch 4/4 (1.1s): Center loss 0.9390, Softmax loss 4.3456, total 5.2870
Train accuracy: 23.84%
Validation mAP (split  5/5) (0.3s): 44.37%
--------------------------------------------------------------------------------
Max. Valid. mAP / Epoch's Valid. mAP: -inf% | 44.37%
Salvando melhor modelo...
--------------------------------------------------------------------------------
Training... Epoch = 2
Epoch 2, Batch 4/4 (0.9s): Center loss 0.8890, Softmax loss 4.2222, total 5.1137
Train accuracy: 30.56%
Validation mAP (split  5/5) (0.3s): 45.82%
--------------------------------------------------------------------------------
Max. Valid. mAP / Epoch's Valid. mAP: 44.37% | 45.82%
Salvando melhor modelo...
---

Epoch 22, Batch 4/4 (0.9s): Center loss 0.5125, Softmax loss 2.4737, total 2.9876
Train accuracy: 93.52%
Validation mAP (split  5/5) (0.2s): 55.72%
--------------------------------------------------------------------------------
Max. Valid. mAP / Epoch's Valid. mAP: 55.68% | 55.72%
Salvando melhor modelo...
--------------------------------------------------------------------------------
Training... Epoch = 23
Epoch 23, Batch 4/4 (1.0s): Center loss 0.5007, Softmax loss 2.3942, total 2.8962
Train accuracy: 94.44%
Validation mAP (split  5/5) (0.2s): 56.86%
--------------------------------------------------------------------------------
Max. Valid. mAP / Epoch's Valid. mAP: 55.72% | 56.86%
Salvando melhor modelo...
--------------------------------------------------------------------------------
Training... Epoch = 24
Epoch 24, Batch 4/4 (0.9s): Center loss 0.4979, Softmax loss 2.2984, total 2.7975
Train accuracy: 94.44%
Validation mAP (split  5/5) (0.2s): 58.02%
--------------------------

Epoch 52, Batch 4/4 (1.0s): Center loss 0.4445, Softmax loss 0.7076, total 1.1520
Train accuracy: 100.00%
Validation mAP (split  5/5) (0.3s): 55.48%
--------------------------------------------------------------------------------
Training... Epoch = 53
Epoch 53, Batch 4/4 (1.0s): Center loss 0.4324, Softmax loss 0.6927, total 1.1251
Train accuracy: 100.00%
Validation mAP (split  5/5) (0.2s): 55.69%
--------------------------------------------------------------------------------
Training... Epoch = 54
Epoch 54, Batch 4/4 (1.0s): Center loss 0.4654, Softmax loss 0.7281, total 1.1934
Train accuracy: 100.00%
Validation mAP (split  5/5) (0.2s): 56.04%
--------------------------------------------------------------------------------
Training... Epoch = 55
Epoch 55, Batch 4/4 (0.9s): Center loss 0.4342, Softmax loss 0.6921, total 1.1263
Train accuracy: 100.00%
Validation mAP (split  5/5) (0.2s): 56.26%
--------------------------------------------------------------------------------
Training...